### Obtaining Insurance Companies Names

In [ ]:
from openai import OpenAI
import json
# Replace with your actual Perplexity AI API key.
API_KEY = ""

ChatCompletion(id='f9b4fd1f-11c8-4713-b599-b3ac5bba6850', choices=[Choice(finish_reason=None, index=0, logprobs=None, message=ChatCompletionMessage(content='{\n  "companies": [\n    "Aetna",\n    "Avera Health Plans",\n    "BCBS Federal Employee Plan",\n    "Cigna",\n    "HealthPartners",\n    "Medica Health Plans",\n    "Medicare",\n    "Sanford Health Plan",\n    "UnitedHealthcare"\n  ]\n}', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None), delta={'role': 'assistant', 'content': ''})], created=1742337842, model='sonar-pro', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=68, prompt_tokens=77, total_tokens=145, completion_tokens_details=None, prompt_tokens_details=None, citation_tokens=7335, num_search_queries=2), citations=['https://www.genedx.com/provider-billing/insurance-billing/', 'https://www.bioreference.com/?s=GeneDx', 'https://www.genedx.com/provider-billing/', 'https://www.genedx.co

In [ ]:


messages = [
    {
        "role": "system",
        "content": (
            "You are an artificial intelligence assistant and you need to "
            "engage in a helpful, detailed, polite conversation with a user."
        ),
    },
    {   
        "role": "user",
        "content": (
            "List the all the medical insurance companies that are in-network with GeneDx."
            "Please output the results in JSON format"
            "The output should have a single fields called companies containing a list of insurance companies. Do not output anything else apart from this list. Do not include any introduction."

        ),
    },
]

client = OpenAI(api_key=API_KEY, base_url="https://api.perplexity.ai")

# chat completion without streaming
response = client.chat.completions.create(
    model="sonar-pro",
    messages=messages,
)
print(response)

# chat completion with streaming
response_stream = client.chat.completions.create(
    model="sonar-pro",
    messages=messages,
    stream=False,
)

In [8]:
companies_list_str = response_stream.choices[0].message.content
companies = json.loads(companies_list_str)["companies"]
companies

['Aetna',
 'Avera Health Plans',
 'BCBS Federal Employee Plan',
 'Cigna',
 'HealthPartners',
 'Medica Health Plans',
 'Medicare',
 'Sanford Health Plan',
 'UnitedHealthcare',
 'Medicaid',
 'Tricare']

### Iteration 2: Obtaining Policy PDFs for Each Insurance Company

In [ ]:
import requests
import json

failed_searches = []

URL = "https://api.perplexity.ai/chat/completions"

# Set up request headers
headers = {
    "Authorization": f"Bearer {API_KEY}",
    "Content-Type": "application/json"
}


# Function to search for PDF policy documents related to genetic testing
def fetch_genetic_testing_pdfs(company):
    """
    Queries Perplexity AI to find PDF files containing genetic testing policies for a given insurance company.

    Args:
        company (str): The name of the insurance company.

    Returns:
        list: A list of PDF URLs found in the search.
    """
    messages = [
        {"role": "system", "content": "You are a helpful AI assistant."},
        {"role": "user", "content": (
            "Find and list all the links to official policy documents that contain genetic testing coverage policies for the insurance company '{company}'."
            "Include both PDF files and webpage URLs if the information is only available on the website."
            "Only include links from official sources such as the insurance company's website or regulatory bodies. "
            "Exclude links from news articles, blog posts, or discussion forums."
            "If the policy is available as a PDF, return the direct PDF link under the key \"pdf_links\"."
            "If the policy is available only as an HTML webpage, return the webpage URL under the key \"webpage_links\". "
            "The response must be strictly in JSON format with two single keys, \"pdf_links\", containing an array of valid PDF URLs, and \"webpage_links\", containing an array of valid webpage URLs. "
            "Do not include any additional text or explanations—only the JSON object."
        )}
    ]

    payload = {
        "model": "sonar-pro",
        "messages": messages,
    }

    response = requests.post(URL, headers=headers, json=payload)

    try:
        response_data = response.json()
        pdf_list_str = response_data["choices"][0]["message"]["content"]
        pdf_links = json.loads(pdf_list_str).get("pdf_links", [])
        return pdf_links
    except (json.JSONDecodeError, KeyError) as e:
        print(f"Error fetching PDFs for {company}: {e}")
        failed_searches.append(company)
        return []

# Dictionary to store results: {company_name: [list of PDF links]}
company_policies = {}

# Loop over each insurance company and fetch PDF links
for company in companies:
    company_policies[company] = fetch_genetic_testing_pdfs(company)

Error fetching PDFs for Aetna: Expecting value: line 1 column 1 (char 0)
Error fetching PDFs for HealthPartners: Expecting value: line 1 column 1 (char 0)


In [5]:
for company in failed_searches:
    company_policies[company] = fetch_genetic_testing_pdfs(company)

In [10]:
# Print the dictionary (clean output)
print(json.dumps(company_policies, indent=4))

{
    "Aetna": [],
    "Avera Health Plans": [
        "https://static.cigna.com/assets/chcp/pdf/coveragePolicies/medical/mm_0518_coveragepositioncriteria_genetic_cancer_syndromes.pdf",
        "https://static.cigna.com/assets/chcp/pdf/coveragePolicies/medical/mm_0052_coveragepositioncriteria_genetic_testing.pdf",
        "https://www.uhcprovider.com/content/dam/provider/docs/public/policies/comm-medical-drug/genetic-testing-hereditary-cancer.pdf"
    ],
    "BCBS Federal Employee Plan": [
        "https://partner.medica.com/-/media/documents/provider/coverage-policies/genetic-testing-general-approach-to-genetic-testing.pdf?la=en&hash=B71D2199B66A946C0C3D8899DAE3CD4F"
    ],
    "Cigna": [
        "https://partner.medica.com/-/media/documents/provider/coverage-policies/genetic-testing-general-approach-to-genetic-testing.pdf?la=en&hash=B71D2199B66A946C0C3D8899DAE3CD4F"
    ],
    "HealthPartners": [],
    "Medica Health Plans": [
        "https://static.cigna.com/assets/chcp/pdf/coverag

In [31]:
#fetch_genetic_testing_pdfs('UnitedHealth Group')